In [1]:
using DelimitedFiles, Plots, Dierckx, QuadGK, SpecialFunctions
using TwoFAST, LaTeXStrings, LsqFit

In [2]:
struct BackgroundData
     z::Vector{Float64}
     conftime::Vector{Float64}
     comdist::Vector{Float64}
     angdist::Vector{Float64}
     lumdist::Vector{Float64}
     D::Vector{Float64}
     f::Vector{Float64}
     ℋ::Vector{Float64}
     ℋ_p::Vector{Float64}

     function BackgroundData(file::String, z_max;
          names = NAMES_BACKGROUND, h = 0.7)
     
          I_redshift = findfirst(x -> x == "z", names)
          I_comdist = findfirst(x -> x == "comov. dist.", names)
     
          data = readdlm(file, comments = true)
     
          N_z_MAX = findfirst(z -> z <= z_max, data[:, I_redshift]) - 1
          com_dist_z_MAX = data[:, I_comdist][N_z_MAX]
          N_2_com_dist_z_MAX = findfirst(s -> s <= 2.0 * com_dist_z_MAX, data[:, I_comdist]) - 1
     
          data_dict = Dict([name => reverse(data[:, i][N_2_com_dist_z_MAX:end])
                            for (i, name) in enumerate(names)]...)
     
          com_H = data_dict["H [1/Mpc]"] ./ h ./ (1.0 .+ data_dict["z"])
          conf_time = data_dict["conf. time [Mpc]"] .* h
          spline_com_H = Spline1D(reverse(conf_time), reverse(com_H); bc = "nearest")
          com_H_p = [Dierckx.derivative(spline_com_H, t) for t in conf_time]
     
          new(
               data_dict["z"],
               conf_time,
               data_dict["comov. dist."] .* h,
               data_dict["ang.diam.dist."] .* h,
               data_dict["lum. dist."] .* h,
               data_dict["gr.fac. D"],
               data_dict["gr.fac. f"],
               com_H,
               com_H_p,
          )
     end
end

In [3]:
NAMES_BACKGROUND = ["z", "proper time [Gyr]", "conf. time [Mpc]", "H [1/Mpc]",
     "comov. dist.", "ang.diam.dist.", "lum. dist.", "comov.snd.hrz.",
     "(.)rho_g", "(.)rho_b", "(.)rho_cdm", "(.)rho_lambda", "(.)rho_ur",
     "(.)rho_crit", "gr.fac. D", "gr.fac. f"];

In [4]:
BD = BackgroundData("../data/WideA_ZA_background.dat", 0.20);

In [ ]:
println(BD.ℋ_p)